In [17]:
from adversarial.defences.adv_classifier import *
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses
from tqdm import tqdm
import time

In [2]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [3]:
import numpy as np

In [4]:
# dataloader
from adversarial.utils import dataloader
from adversarial.attacks import random_attack

In [5]:
BATCH_SIZE = 32
EPSILONS = np.arange(0.01, 0.5, step=0.02)
ETA = 1e-4
EPOCHS = 10

In [6]:
train_data_loader = dataloader.DataLoader('./data', BATCH_SIZE, training=True)
test_data_loader = dataloader.DataLoader('./data', BATCH_SIZE, training= False)

In [7]:
from adversarial.defences import EnsembleModel
from adversarial.models import *

random_attack(앙상블 모델 넣을건데, x_image, adversarial_label, epsilon은 배열로)  
epsilon = np.arange(0.01,0.5,0.02)  
random_attack은 adversarial image를 리턴해준다. 

In [8]:
target_model= AdvClassifier()

In [9]:
# train

criterion = losses.CategoricalCrossentropy(label_smoothing=0.2)
optimizer = tf.keras.optimizers.Adam(learning_rate = ETA)

train_losses = tf.keras.metrics.CategoricalCrossentropy()
train_metrics = tf.keras.metrics.CategoricalAccuracy()

test_losses = tf.keras.metrics.CategoricalCrossentropy()
test_metrics = tf.keras.metrics.CategoricalAccuracy()

In [10]:
ensemble_model = EnsembleModel([
    VGG19, VGG16, MobileNet, MobileNetV2, ResNet50, ResNet50V2
])

In [11]:
labels = [[0,1] ,[1,0]]
    
y_batch = []
    
    
for label in labels : 
    for time in range(BATCH_SIZE) :
        y_batch.append(label)

y_tensor = tf.convert_to_tensor(y_batch)

In [12]:
def train_step(images, adv_labels):

    adv, _, success = random_attack(ensemble_model, images, adv_labels, EPSILONS)
    
    x_batch = tf.concat([images, adv], axis=0)
    y_batch = y_tensor
    with tf.GradientTape() as tape:
        outputs = target_model(x_batch, training = True)
      #  print('y_batch : ' ,y_batch)
       # print('outputs : ' ,outputs)
        loss = criterion(y_batch, outputs)
    print('batch loss : {0}'.format(loss.numpy()), end = '\r')
    train_losses.update_state(y_batch, outputs)
    train_metrics.update_state(y_batch, outputs)
        
    grads = tape.gradient(loss, target_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, target_model.trainable_variables))

In [13]:
def test_step(images, labels, adv_labels) :
    adv,_, success = random_attack(ensemble_model, images, adv_labels, EPSILONS)
    x_batch = tf.concat([images, adv], axis=0)
    y_batch = y_tensor
    #y_batch = tf.concat([tf.zeros(16, dtype = tf.int32),
     #                    tf.ones(16, dtype = tf.int32)], 
      #                   axis = 0)
    
    predictions = target_model(x_batch, training= False)
    loss = tf.keras.losses.binary_crossentropy(y_batch, predictions)
    test_losses.update_state(y_batch, predictions)
    test_metrics.update_state(y_batch, predictions)

In [19]:
def train():

    train_loss_list = []
    test_loss_list = []

    for e in range(EPOCHS):
        start = time.time()
        for x_batch, y_batch, adv_batch in iter(train_data_loader):
            train_step(x_batch, adv_batch)

        for x_batch, y_batch, adv_batch in iter(test_data_loader):
            test_step(x_batch, y_batch, adv_batch)


        train_loss = train_losses.result()
        train_acc = train_metrics.result()

        test_loss = test_losses.result()
        test_acc = test_metrics.result()

        train_loss_list.append(train_loss)
        test_loss_list.append(test_loss)

        train_losses.reset_states()
        train_metrics.reset_states()
        test_losses.reset_states()
        test_metrics.reset_states()
        end = time.time()
        print(f"Epochs {e+1}/{EPOCHS}, train loss: {train_loss:.8f}, train acc: {train_acc:.4f}, test loss: {test_loss:.8f}, test acc: {test_acc:.4f}, time_cost: {end-start}s")

In [21]:
train()

Epochs 1/10, train loss: 0.62176311, train acc: 0.6613, test loss: 0.54507875, test acc: 0.6855, time_cost: 2609.6581983566284s
Epochs 2/10, train loss: 0.58860153, train acc: 0.6587, test loss: 0.53270948, test acc: 0.7109, time_cost: 2770.6045999526978s
Epochs 3/10, train loss: 0.55395162, train acc: 0.6714, test loss: 0.56398332, test acc: 0.6523, time_cost: 3454.5851106643677s
Epochs 4/10, train loss: 0.55065924, train acc: 0.6628, test loss: 0.70388210, test acc: 0.5293, time_cost: 3310.0244274139404s
Epochs 5/10, train loss: 0.51733398, train acc: 0.6850, test loss: 0.55788344, test acc: 0.6436, time_cost: 2873.4723043441772s
Epochs 6/10, train loss: 0.53949726, train acc: 0.6571, test loss: 0.45360413, test acc: 0.7227, time_cost: 2622.358393907547s
Epochs 7/10, train loss: 0.51027137, train acc: 0.6813, test loss: 0.48777840, test acc: 0.6855, time_cost: 3087.1960372924805s
Epochs 8/10, train loss: 0.49250671, train acc: 0.6935, test loss: 0.45618841, test acc: 0.7207, time_cos

In [22]:
train()

Epochs 1/10, train loss: 0.51750517, train acc: 0.6736, test loss: 0.57008600, test acc: 0.6309, time_cost: 3243.5787994861603s
Epochs 2/10, train loss: 0.49869192, train acc: 0.6937, test loss: 0.57917112, test acc: 0.6172, time_cost: 2432.589989423752s
Epochs 3/10, train loss: 0.50031179, train acc: 0.6885, test loss: 0.52491742, test acc: 0.6758, time_cost: 2649.0924146175385s
Epochs 4/10, train loss: 0.52716410, train acc: 0.6734, test loss: 0.41781813, test acc: 0.7646, time_cost: 3073.472977876663s
Epochs 5/10, train loss: 0.51241469, train acc: 0.6862, test loss: 0.44113183, test acc: 0.7549, time_cost: 3200.534516096115s
Epochs 6/10, train loss: 0.49848786, train acc: 0.6974, test loss: 0.45478064, test acc: 0.7441, time_cost: 2988.365292072296s
Epochs 7/10, train loss: 0.50691944, train acc: 0.6886, test loss: 0.58776140, test acc: 0.6172, time_cost: 2562.6712470054626s
Epochs 8/10, train loss: 0.48853531, train acc: 0.7088, test loss: 0.40575883, test acc: 0.7607, time_cost: 

In [23]:
train()

Epochs 1/10, train loss: 0.52093798, train acc: 0.6811, test loss: 0.52138489, test acc: 0.6914, time_cost: 3488.5986235141754s
Epochs 2/10, train loss: 0.49313843, train acc: 0.7070, test loss: 0.47310925, test acc: 0.7168, time_cost: 3222.2192149162292s
Epochs 3/10, train loss: 0.49812859, train acc: 0.7057, test loss: 0.59254462, test acc: 0.6436, time_cost: 3079.3095376491547s
Epochs 4/10, train loss: 0.50879407, train acc: 0.6973, test loss: 0.39965388, test acc: 0.7803, time_cost: 2875.8597536087036s
Epochs 5/10, train loss: 0.50002253, train acc: 0.7075, test loss: 0.44758257, test acc: 0.7793, time_cost: 2754.562947511673s
Epochs 6/10, train loss: 0.51897883, train acc: 0.6903, test loss: 0.41976553, test acc: 0.7930, time_cost: 3273.222067832947s
Epochs 7/10, train loss: 0.51053315, train acc: 0.6987, test loss: 0.52199429, test acc: 0.7051, time_cost: 2905.5555543899536s
Epochs 8/10, train loss: 0.48209170, train acc: 0.7197, test loss: 0.66398978, test acc: 0.5459, time_cost

In [24]:
save_custom_weights(target_model,'./model/adv_classifier_6.5.pkl')

NameError: name 'save_custom_weights' is not defined

In [ ]:
def load_custom_weights(model, path):
    with open(path, "rb") as f:
        data = pickle.load(f)
        model.set_weights(data)

def save_custom_weights(model, path):
    with open(path, "wb") as f:
        pickle.dump(model.get_weights(), f)